In [1]:
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"    
import tensorflow as tf
import sys
sys.path.append("..")
from Vocabulary import *
import math
import numpy as np

# Next block contains all parameters, which need to be set

In [2]:
#Configure this block
embedding_path = r"D:\\mixed\\"
embedding_name = 'ner_mixed_150w'
outlocation = "D:\\MixS1\\"
New_emb_Name_path = outlocation+embedding_name + '_combined'#later added with String _strategyX
unfiltered_vocab = Vocabulary()
unfiltered_vocab.load(r"C:\Users\weso\Desktop\gauntlet\vocabs\unfilteredvocab_ner_mixed")
dimensions = 500

In [3]:
import numpy as np
import tensorflow as tf
id_dict = {}
word_dict = {}

matrix = []
full_path = os.path.join(embedding_path,embedding_name)
with open(full_path, 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

617537


In [4]:
#find all variants of a word, without the tag rep
def group_all_words(id_dict):
    groups = {}
    for word in id_dict:
        #print(word)
        if is_tagged_word(word):
            #print('is tagged')
            baseword = word.split(chr(4))[0]
            #print(baseword)
            if baseword in groups:
                groups[baseword].append(word)
            else:
                groups[baseword] = [word]
        elif word.startswith(chr(4)):#exclude tags
            pass
        else:
            if word in groups:
                groups[word].append(word)
            else:
                groups[word] = [word]
    return groups

def is_tagged_word(word):
    return not word.startswith(chr(4)) and str(chr(4)) in word

groups = group_all_words(id_dict)
print(len(groups))
print(groups['duck'])
print(groups['the'])
print(groups['bank'])
print('---------')
print(str(groups)[:100])
print(groups["splashdown"])

260468
['duck\x04org', 'duck', 'duck\x04work_of_art', 'duck\x04gpe', 'duck\x04fac', 'duck\x04person', 'duck\x04loc', 'duck\x04']
['the\x04money', 'the\x04work_of_art', 'the\x04person', 'the\x04law', 'the\x04gpe', 'the\x04fac', 'the', 'the\x04event', 'the\x04time', 'the\x04product', 'the\x04date', 'the\x04loc', 'the\x04org', 'the\x04', 'the\x04norp', 'the\x04cardinal', 'the\x04percent', 'the\x04quantity']
['bank\x04date', 'bank\x04fac', 'bank\x04gpe', 'bank\x04org', 'bank\x04event', 'bank', 'bank\x04loc', 'bank\x04work_of_art', 'bank\x04', 'bank\x04person']
---------
{'ppv': ['ppv\x04', 'ppv', 'ppv\x04org'], 'ingraham': ['ingraham', 'ingraham\x04person', 'ingraham\x
['splashdown\x04', 'splashdown']


# helper

In [5]:
def get_count(word):
    return unfiltered_vocab.word_frequency[word]

In [6]:
def save_new_embedding(embedding,path):
    with open(path,'w+',encoding='utf8') as file:
        for id,word in enumerate(embedding):
            if(id % 1000 == 0):
                print(id,len(embedding))
            
            file.write(word)
            vector = embedding[word]
            for coord in vector:
                file.write(' '+str(coord))
            file.write('\n')

# Strategy 1 weighted Sum

In [7]:
#requires tag rep
def mult_strat_weighted(word_groups,normalise_before_mult):
    emb = {}
    
    if(normalise_before_mult):
        matrix_copy = tf.nn.l2_normalize(matrix,axis = 1)
    else:
        matrix_copy = matrix
        
    for word_key in groups:

        base_count = 0 # total apearance of the baseword
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            if chr(4) in word:
                if(word.startswith(chr(4))):#no rep for tags
                    continue
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                #id_tag  = id_dict[chr(4)+tag]
                #emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] * matrix_copy[id_tag])
                emb_sum += (get_count(word)/base_count) * (matrix_copy[id_word])
            else:#only in mixed embedding add the base unscaled
                emb_sum += (get_count(word)/base_count) * matrix_copy[id_dict[word]]
        emb[baseword] = emb_sum
    return emb
    

In [8]:
emb1 = mult_strat_weighted(groups,False)
save_new_embedding(emb1,New_emb_Name_path+"_Strat1")

0 195699
1000 195699
2000 195699
3000 195699
4000 195699
5000 195699
6000 195699
7000 195699
8000 195699
9000 195699
10000 195699
11000 195699
12000 195699
13000 195699
14000 195699
15000 195699
16000 195699
17000 195699
18000 195699
19000 195699
20000 195699
21000 195699
22000 195699
23000 195699
24000 195699
25000 195699
26000 195699
27000 195699
28000 195699
29000 195699
30000 195699
31000 195699
32000 195699
33000 195699
34000 195699
35000 195699
36000 195699
37000 195699
38000 195699
39000 195699
40000 195699
41000 195699
42000 195699
43000 195699
44000 195699
45000 195699
46000 195699
47000 195699
48000 195699
49000 195699
50000 195699
51000 195699
52000 195699
53000 195699
54000 195699
55000 195699
56000 195699
57000 195699
58000 195699
59000 195699
60000 195699
61000 195699
62000 195699
63000 195699
64000 195699
65000 195699
66000 195699
67000 195699
68000 195699
69000 195699
70000 195699
71000 195699
72000 195699
73000 195699
74000 195699
75000 195699
76000 195699
77000 195699

# Strategy 3 AddTagRep

In [ ]:
#requires tag rep
def add_tag_rep_strat(word_groups,normalise_before_mult):
    emb = {}
    
    if(normalise_before_mult):
        matrix_copy = tf.nn.l2_normalize(matrix,axis = 1)
    else:
        matrix_copy = matrix
        
    for word_key in groups:
        
        base_count = 0 # total apearance of the baseword
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            c = get_count(word)
            base_count += c
        
        emb_sum = 0
        for word in word_groups[word_key]:
            if(word.startswith(chr(4))):#no rep for tags
                continue
            if chr(4) in word:
                baseword ,tag  = word.split(chr(4))
                id_word = id_dict[word]
                #id_tag  = id_dict[chr(4)+tag]
                #emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] * matrix_copy[id_tag])
                emb_sum += (get_count(word)/base_count)*(matrix_copy[id_word] + matrix_copy[id_tag])
            else:#only in mixed embedding add the base unscaled
                emb_sum += matrix_copy[id_dict[word]] * (get_count(word)/base_count)
        emb[baseword] = emb_sum
    return emb

In [ ]:
emb3 = add_tag_rep_strat(groups,False)
save_new_embedding(emb3,New_emb_Name_base+"_Strat3")